In [13]:
import pandas as pd
import pickle
import datetime

In [2]:
import sys
sys.path.insert(0, './utils/')

import test as t

In [17]:
import importlib
importlib.reload(t)

<module 'test' from 'E:\\github\\lending_club\\lending_club_loans_default_prediction\\./utils\\test.py'>

In [4]:
data_path = './data/loan.csv'

df = pd.read_csv(data_path)

C:\Users\xzhou\AppData\Local\Temp\ipykernel_8172\1420088683.py:3: DtypeWarning: Columns (19,55) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(data_path)


In [5]:
# Assign labels based on loan status

excluded = {'Issued', 'Does not meet the credit policy. Status:Charged Off',
            'Does not meet the credit policy. Status:Fully Paid'}
positive = {'Current', 'Fully Paid'}
negative = {'Charged Off', 'Default', 'In Grace Period', 
       'Late (16-30 days)', 'Late (31-120 days)'}

def assign_label(status):
    """
    Classify loan staus based on description
    """
    try: 
        status=status.strip()
        if status in positive:
            return "positive"
        elif status in negative:
            return "negative"
        else: 
            return "excluded"
    except: 
        return "excluded"

In [6]:
df['label'] = df['loan_status'].apply(lambda x: assign_label(x))

In [7]:
# Drop ids, and loan status to prevent info leakage
cols_to_drop = ['id', 'member_id', 'loan_status']
df_id_removal = df.drop(columns = cols_to_drop)

In [8]:
# Only keep loans with good or bad status
df_label = df_id_removal[df_id_removal['label']!='excluded']
df_label.label.unique()

array(['positive', 'negative'], dtype=object)

In [9]:
features = df_label.drop('label', axis=1)
labels = df_label['label']

In [16]:
features.shape

(876170, 71)

In [34]:
features.columns.tolist()

['loan_amnt',
 'funded_amnt',
 'funded_amnt_inv',
 'term',
 'int_rate',
 'installment',
 'grade',
 'sub_grade',
 'emp_title',
 'emp_length',
 'home_ownership',
 'annual_inc',
 'verification_status',
 'issue_d',
 'pymnt_plan',
 'url',
 'desc',
 'purpose',
 'title',
 'zip_code',
 'addr_state',
 'dti',
 'delinq_2yrs',
 'earliest_cr_line',
 'inq_last_6mths',
 'mths_since_last_delinq',
 'mths_since_last_record',
 'open_acc',
 'pub_rec',
 'revol_bal',
 'revol_util',
 'total_acc',
 'initial_list_status',
 'out_prncp',
 'out_prncp_inv',
 'total_pymnt',
 'total_pymnt_inv',
 'total_rec_prncp',
 'total_rec_int',
 'total_rec_late_fee',
 'recoveries',
 'collection_recovery_fee',
 'last_pymnt_d',
 'last_pymnt_amnt',
 'next_pymnt_d',
 'last_credit_pull_d',
 'collections_12_mths_ex_med',
 'mths_since_last_major_derog',
 'policy_code',
 'application_type',
 'annual_inc_joint',
 'dti_joint',
 'verification_status_joint',
 'acc_now_delinq',
 'tot_coll_amt',
 'tot_cur_bal',
 'open_acc_6m',
 'open_il_6m',


In [35]:
features['next_pymnt_d'].head()

0         NaN
1         NaN
2         NaN
3         NaN
4    Feb-2016
Name: next_pymnt_d, dtype: object

In [25]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import StandardScaler, FunctionTransformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.feature_extraction import DictVectorizer
from sklearn.decomposition import PCA
from scipy.stats import pearsonr
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder

In [47]:
def preprocessing_pipeline(df):
    # Define the pipeline steps
    date_pipeline = t.LoggingPipeline([
    ('to_days', t.DateTransformer())
])

    numerical_pipeline = t.LoggingPipeline([
        ('imputer', SimpleImputer(strategy='median')),
        ('winsorizer', t.ExtremeValuesNumericalHandler()),
        # ('scaler', StandardScaler()),
    ])

#     categorical_pipeline = LoggingPipeline([
#         ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
#         ('onehot', OneHotEncoder(handle_unknown='ignore'))
#     ])
    # Get the column names
    date_columns = [col for col in df.columns if 'date' in col.lower()]
    numerical_columns = df.select_dtypes(include=['number']).columns.tolist()
    categorical_columns = df.select_dtypes(include=['object']).columns.tolist()

    # Combine the pipelines
    full_pipeline = ColumnTransformer(transformers=[
        ('date', date_pipeline, date_columns),
        ('num', numerical_pipeline, numerical_columns),
        # ('cat', categorical_pipeline, categorical_columns)
    ])

    return full_pipeline

In [48]:
pipeline = preprocessing_pipeline(features)

In [49]:
processed_features = pipeline.fit_transform(features)

TypeError: 'NoneType' object is not iterable

In [32]:
pd.DataFrame(processed_features).columns

RangeIndex(start=0, stop=0, step=1)

In [ ]:
test_features = df_test.drop('label', axis=1)
processed_test_features = pipeline.transform(test_features)